### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Psoriasis/GSE182740'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
is_gene_available = True  # The dataset is described to have gene expression profiles

# Determine the rows for each variable based on the sample characteristics dictionary
trait_row = 1 if 'disease: Psoriasis' in sample_characteristics[1] else None
age_row = None  # There is no age data in the sample characteristics
gender_row = None  # There is no gender data in the sample characteristics

# Define conversion functions for each variable
def convert_trait(value):
    try:
        val = value.split(':')[1].strip()
        if val == 'Psoriasis':
            return 1
        elif val == 'Atopic_dermatitis' or val == 'Mixed' or val == 'Normal_skin':
            return 0
        else:
            return None
    except:
        return None

def convert_age(value):
    # No age data available, hence return None
    return None

def convert_gender(value):
    # No gender data available, hence return None
    return None

# Save metadata
save_cohort_info('GSE182740', './preprocessed/Psoriasis/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and process clinical features if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Psoriasis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Psoriasis/trait_data/GSE182740.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
